In [30]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
cache = None
api = None
def _download():
    global cache, api
    if api is None:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    cache = api.download_issues(project_id="pbssoftware")
def download(force=False):
    global cache
    if cache is None or force:
        _download()

In [3]:
def relevant(i):
    if i.sprint_milestone != "3.0" or i.tracker != "Specification":
        return False
    if i.status in ("Rejected", "Closed"):
        return False
    return True

def filter_issues():
    global cache, issues_30
    issues_30 = [i for i in cache.values() if relevant(i)]
    print("Filtered %d relevant issues" % len(issues_30))
    
def get_issues(recache=False):
    download(recache)
    filter_issues()
    return issues_30.copy()

In [15]:
for i in get_issues():
    exp = "*+FRS%d.1+*" % i.id
    d = i.description
    if d.find(exp) == -1:
        print(i.id)

3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
989
3083
3087
3090
3093
3101
3102
3106
3107
3108
3109
3114
3118
1885
3120
3122
3124
3130
3131
3132
3133
3134
3135
3136
2115
3141
3145
2231
3150
3152
3153
3154
3155
3156
3157
3158
3159
3161
3163
3164
3165
3166
3167
3168
3175
3176
3177
3178
3183
3186
3188
3189
3191
3192
3194
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
1902
3223
3224
3225
3226
3229
3230
3231
3233
3234
3235
3236
3237
3238
3240
3242
3244
3245
3246
3247
3248
3249
3250
3252
3253
3254
3255
3258
3259
3260
3261
3264
3266
3267
3268
3269
3272
2251
3280
3287
3288
3289
3290
1262
2303
2304
2309
2316
2320
2321
1953
2367
396
2394
2395
2404
2415
2445
2458
1263
2460
2230
420
2469
2476
2484
1583
2500
2502
2516
2522
1073
2592
2596
2626
2031
2660
2669
2681
2684
2703
2711
2712
2713
2715
2716
1694
2725
2726
2733
2734
2735
2747
2752
2757
2758
2767
2769
1749
1754
2780
1759
1572
2787
2789
2794
2796
2801
2805
1783
2808
2809
2810
2812
2814
281

In [33]:
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
for i in get_issues():
    if not frs_re.search(i.description):
        if i.id == 3194:  # URS issue
            continue 
        if web_frs_re.search(i.description):
            print("Goddamit Christian: %d" % i.id)
        else:
            print("Goddamnit Nathan: %d" % i.id)

In [36]:
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
n=0
noncompliant = []; add = noncompliant.append
for i in get_issues():
    exp = "*+FRS%d.1+*" % i.id
    exp2 = "*+FRS%d.0+*" % i.id
    if i.description.find(exp) == -1 and i.description.find(exp2) == -1:
        if not web_frs_re.search(i.description):
            add("Goddamnit Nathan: %d" % i.id)
        
print("Found %d noncompliant issues" % len(noncompliant))
for msg in noncompliant:
    print(msg)

Found 80 noncompliant issues
Goddamnit Nathan: 989
Goddamnit Nathan: 2115
Goddamnit Nathan: 3176
Goddamnit Nathan: 3194
Goddamnit Nathan: 1902
Goddamnit Nathan: 3225
Goddamnit Nathan: 3226
Goddamnit Nathan: 3288
Goddamnit Nathan: 3289
Goddamnit Nathan: 1262
Goddamnit Nathan: 2304
Goddamnit Nathan: 2316
Goddamnit Nathan: 2320
Goddamnit Nathan: 2321
Goddamnit Nathan: 2367
Goddamnit Nathan: 2404
Goddamnit Nathan: 2445
Goddamnit Nathan: 2458
Goddamnit Nathan: 1263
Goddamnit Nathan: 2460
Goddamnit Nathan: 420
Goddamnit Nathan: 2476
Goddamnit Nathan: 1583
Goddamnit Nathan: 2502
Goddamnit Nathan: 2516
Goddamnit Nathan: 2522
Goddamnit Nathan: 1073
Goddamnit Nathan: 2596
Goddamnit Nathan: 2626
Goddamnit Nathan: 2734
Goddamnit Nathan: 2757
Goddamnit Nathan: 2769
Goddamnit Nathan: 1754
Goddamnit Nathan: 2780
Goddamnit Nathan: 1759
Goddamnit Nathan: 2787
Goddamnit Nathan: 2796
Goddamnit Nathan: 2805
Goddamnit Nathan: 2808
Goddamnit Nathan: 2819
Goddamnit Nathan: 2825
Goddamnit Nathan: 2828
Goddamn

In [6]:
def search(recache=False):
    compliant_search = re.compile(r"[\+\*]{1,2}(?:3\.0Web)?FRS[\d\.]*\:?[\+\*]{1,2}").search
    frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
    web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
    n=0
    def compliant(i):
        return i.id == 3194 or compliant_search(i.description)
    noncompliant = []; add = noncompliant.append
    for i in get_issues(recache):
        if not compliant(i):
            #if not web_frs_re.search(i.description):
                add("Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    for msg in noncompliant:
        print(msg)

In [7]:
search()

Filtered 81 relevant issues
Found 0 noncompliant issues


In [35]:
compliant_search = re.compile(r"^[\+\*]{1,2}(?:3\.0Web)?FRS\d+\:?[\+\*]{1,2}", re.MULTILINE).search
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")

def compliant_description(d):
    return compliant_search(d) is not None
def compliant(i):
    return (i.id == 3194 or compliant_description(i.description))

def search(recache=False):
    compliant_search = re.compile(r"^[\+\*]{1,2}(?:3\.0Web)?FRS\d+\:?[\+\*]{1,2}").search
    frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
    web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
    n=0
    noncompliant = []
    def add(id, msg):
        noncompliant.append((id, msg))
    for i in get_issues(recache):
        if not compliant(i):
            #if not web_frs_re.search(i.description):
                add(i.id, "Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    noncompliant.sort(key=lambda arg:arg[0])
    for _, msg in noncompliant:
        print(msg)
search(True)

Filtered 78 relevant issues
Found 0 noncompliant issues


In [33]:
import clipboard
print(compliant_description(clipboard.paste()))

True


In [155]:
compliant_search = re.compile(r"^[\+\*]{1,2}(?:3\.0Web)?FRS\d+\:?[\+\*]{1,2}", re.MULTILINE).search
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")

toplevel_match = re.compile(r"^([\+\*]{2})(FRS\d+)([\+\*]{2})(.*)$").match
header_match = re.compile(r"^([\+\*]{2})([\w]+[^\:]+)(\:)?([\+\*]{2})(\:)?(.*)$").match
valid_headers = ["User Requirement", "James's Notes", "Original FRS", "URS","Action", "User Requirements","Notes", "Note", "Implementation","Implementation Details","Current Behavior", "Background"]
subitem_match = re.compile(r"^([\+\*]+)\s\*(FRS)([\d\.]+)\:\*(.+)$").match
list_match = re.compile(r"([\#\*]+)\s(.*)").match
img_match = re.compile(r"^!(.+)!$").match
captions = ["Image"]
caption_match = re.compile(r"\*((?:%s) \d+)\*\:(.+)$" % "|".join(captions)).match
note_match = re.compile(r"\*?(Note|Notes)\:?\*\:?").match
maybe_frs = re.compile(r"(FRS\d[\d\.]+)").search
webfrs_match = re.compile(r"^[\*\+]+(3\.0WebFRS\d+)[\*\+]+").match

TOPLEVEL = 0
HEADER = 1
SUBITEM = 2
OTHER = 3
LISTITEM = 4
EMPTY = 5
IMG = 6
CAPTION = 7
NOTE = 8
MAYBE_FRS = 9
WEB_FRS = 10

def compliant_line(i, l):
    if not l.strip():
        return True, EMPTY, ""
    
    m = toplevel_match(l)
    if m:
        pre, frs, post, other = m.groups()
        if pre != "*+" or post != "+*": return False, TOPLEVEL, "Toplevel: pre != post: %r" % l
        if frs[3:] != str(i.id): return False, TOPLEVEL, "FRS ID mismatch"
        if other.strip(): return False, TOPLEVEL, "Tail: %r" % l
        return True, TOPLEVEL, ""
    
        
    if webfrs_match(l):
        return True, WEB_FRS, ""
    
    m = header_match(l)
    if m:
        pre, header, icol, post, ocol, other = m.groups()
        if pre != post[::-1]: return False, HEADER, "Header: pre != post: %r and %r" % (pre, post)
        if header not in valid_headers: 
            # Special cases
            if header.startswith("Implementation"): return True, HEADER, ""
            if header.startswith("Original FRS"): return True, HEADER, ""
            return False, HEADER, "Invalid header: %r" % header
        if other.strip(): print("Warning, found non-empty other in header")
        return True, HEADER, ""
    
    m = subitem_match(l)
    if m:
        ind, frs, nums, other = m.groups()
        if len(ind) != len(nums.split("."))-1: return False, SUBITEM, "Wrong Indent"
        if not other: return False, SUBITEM, "No text for subitem"
        return True, SUBITEM, ""
    
    m = list_match(l)
    if m:
        ind, other = m.groups()
        c = ind[0]
        if not all(ch == c for ch in ind): return False, LISTITEM, "wtf"
        return True, LISTITEM, ""
    
    m = img_match(l)
    if m:
        return True, IMG, ""
    
    m = caption_match(l)
    if m:
        return True, CAPTION, ""
    
    m = note_match(l)
    if m:
        return True, NOTE, ""

    
    if maybe_frs(l):
        # special case
        if l.startswith("*Original FRS"): return True, HEADER, ""
        return False, MAYBE_FRS, "MaybeFRS: %r" % l
    

    
    return True, OTHER, "Other: %r" % l

def consume_pre(it):
    level = 1
    while level > 0:
        line = next(it)
        if "<pre>" in line:
            level += 1
        elif "</pre>" in line:
            level -= 1

def compliant_description(i, d):
    rv = True
    it = iter(d.splitlines())
    while True:
        line = next(it, None)
        if line is None:
            break
        if "<pre>" in line:
            line = consume_pre(it)
            if line is None:
                break
            
        comp, typ, msg = compliant_line(i, line)
        if not comp:
            print(i.id, msg)
            print()
            rv = False
    return rv
    
def compliant(i):
    return (i.id == 3194 or compliant_description(i, i.description))

def search(recache=False):
    n=0
    noncompliant = []
    def add(id, msg):
        noncompliant.append((id, msg))
    for i in get_issues(recache):
        if not compliant(i):
            add(i.id, "Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    noncompliant.sort(key=lambda arg:arg[0])
    for _, msg in noncompliant:
        print(msg)
search(False)

Filtered 78 relevant issues
2115 Tail: '*+FRS2115+*: All Web UI Login Events must include the user who logged in.  No WebUI login will show "Unknown User."'

1902 Tail: '*+FRS1902+*: Calibrations shall be rejected if m or b is Inf or -Inf'

3225 Tail: '*+FRS3225+* Harvest Valve'

1073 MaybeFRS: 'All FRS with the form "FRS1073.0X" have actions which are in the Shell'

1073 MaybeFRS: 'All FRS with the form "FRS1073.1X" use the Webservice Event to DB sub vi'

1073 MaybeFRS: 'All FRS with the form "FRS1073.13X" use the Webservice Event to DB sub vi and are in the Webservice Command Handler'

2828 Tail: '*+FRS2828+* The windows computer shall be configured with the following accounts:'

2875 Tail: '*+FRS2875+* Database History file'

2877 Tail: '*+FRS2877+*: Implement O2 Gas Totalizing for Mag Drive Bioreactors'

2884 Tail: '*+FRS2884+* Appearance'

2888 Tail: '*+FRS2888+* The server should NOT do the following:'

2890 Tail: '*+FRS2890+* A utility shall log the contents of the following fil

In [128]:
m1 = re.compile(r"\*\+(.+)\+\*").match
m2 = re.compile(r"(FRS\d+)").search
m3 = None
for i in get_issues():
    d = i.description
    for l in d.splitlines():
        if not m1(l):
            if m2(l):
                try:
                    a, _= l.split(" ", 1)
                except:
                    pass
                else:
                    if a.count("*") != len(a):
                        print(l)
         

Filtered 78 relevant issues
All FRS with the form "FRS1073.0X" have actions which are in the Shell
All FRS with the form "FRS1073.1X" use the Webservice Event to DB sub vi
All FRS with the form "FRS1073.13X" use the Webservice Event to DB sub vi and are in the Webservice Command Handler
*Original FRS1073.01*
*Original FRS1073.02*
*Original FRS1073.03*
*Original FRS1073.11*
*Original FRS1073.1304*
*Original FRS1073.1305*
*Original FRS1073.1306*
*Original FRS1073.1307*
*Original FRS1073.1308*
*Original FRS1073.1309*
*Original FRS1073.1310*
*Original FRS1073.1311*
*Original FRS1073.1312*
*Original FRS1073.1313*
*Original FRS1073.1314*
*Original FRS1073.1315*
*Original FRS1073.1316*


In [160]:
import os
toplevel_match = re.compile(r"^([\+\*]{2})(FRS\d+)([\+\*]{2})(.*)$").match
header_match = re.compile(r"^([\+\*]{2})([\w]+[^\:]+)(\:)?([\+\*]{2})(\:)?(.*)$").match

def extract_descr(d):
    it = iter(d.splitlines())
    frs = []
    m = None
    while not m:
        line = next(it, None)
        if line is None:
            return frs
        m = toplevel_match(line)
        if m:
            while True:
                frs.append(line)
                line = next(it, None)
                if line is None:
                    break
                if header_match(line):
                    break
    return frs
                

def write_frs(f, frs):
    for i, lines in frs:
        #f.write("=====================\n")
        f.write("*+Issue #%d+*: %s\n" % (i.id, i.subject))
        #f.write("*+Subject:+* %s\n" % i.subject)
        #f.write("=====================\n\n")
        #f.write("\n".join(lines))
        for line in lines:
            f.write(">"+line+"\n")
        f.write("\n\n")
    
def extract_frs():
    frs = []
    for i in get_issues():
        lines = extract_descr(i.description)
        if not lines: continue
        frs.append((i, lines))
    frs.sort(key=lambda v:v[0].id)
    with open("full_frs.txt", "w") as f:
        write_frs(f, frs)
        
    
extract_frs()
with open("full_frs.txt", 'r') as f:
    clipboard.copy(f.read())
os.startfile("full_frs.txt")
    
        
        

Filtered 78 relevant issues


In [134]:
header_match = re.compile(r"^([\+\*]{2})([\w]+[^\:]+)(\:)?([\+\*]{2})(\:)?(.*)$").match
header_match("*+Implementation of FRS3226.3+*")

<_sre.SRE_Match object; span=(0, 31), match='*+Implementation of FRS3226.3+*'>

In [150]:
toplevel_match = re.compile(r"^([\+\*]{2})(FRS\d+)([\+\*]{2})(.*)$").match
subitem_match = re.compile(r"^([\+\*]+)\s\*(FRS)([\d\.]+)\:\*(.+)$").match

def descr_nums(desc):
    nums = []
    for l in desc.splitlines():
        m = toplevel_match(l)
        if m:
            _, frs, _, _ = m.groups()
            nums.append(frs)
            continue
        m = subitem_match(l)
        if m:
            _, frs, ns, _ = m.groups()
            nums.append(frs+ns)
            continue
    return nums

def all_frs_numbers():
    nums = []
    for i in get_issues():
        nums.extend(descr_nums(i.description))
    nums.sort(key=lambda v: tuple(int(i) for i in v[3:].split(".")))
    return nums
nums=all_frs_numbers()
nums = set(nums)

with open("full_frs.txt", 'r') as f:
    txt = f.read()
nums2 = descr_nums(txt)
nums2 = set(nums2)

print(nums == nums2)
    
                
                

Filtered 78 relevant issues
True


In [151]:
with open("full_frs.txt", 'r') as f:
    clipboard.copy(f.read())